In [208]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC

Reading data:

In [119]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set



In [120]:
train_set, test_set, val_set = read_data("data5", "train_data", "test_data", "valid_data")

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [121]:
train_set_np = train_set.values
test_set_np = test_set.values
val_set_np = val_set.values

In [122]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [123]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_set_np, test_set_np, val_set_np)

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [124]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [125]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

# Labels

In [126]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [127]:
 train_labels, test_labels, val_labels = np_labeliser(train_set_nb, test_set_nb, val_set_nb, 22)

# Feature Selection

In [128]:
def np_featuriser(train, test, val,feature_lsit):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val

In [272]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [275]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
#print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.08:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove

[ 0.01470311  0.0074998   0.04611609  0.04593729  0.08357463  0.02805854
  0.30318339  0.34438477  0.31151492  0.02153433  0.00746457  0.12981515
  0.16454504  0.05304241  0.08644591  0.33475251  0.16157611  0.15386772
  0.03772989  0.05008168  0.05688609  0.00259443  0.75265577  1.        ]
Most valuable features: 
4 0.0835746288741 FGA
6 0.30318339431 3PM
7 0.344384768543 3PA
8 0.311514923218 3P%
11 0.129815150674 FT%
12 0.164545043096 OFF
14 0.0864459081968 TRB
15 0.334752506224 AST
16 0.16157611031 STL
17 0.15386772173 BLK
23 0.752655766218 W
24 1.0 H
[0, 1, 2, 3, 5, 9, 10, 13, 18, 19, 20, 21, 22]


In [276]:
feature_list = [0, 1, 2, 3, 4, 6, 10, 11, 14, 19, 20, 21]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [146]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
train = train_features_nb

print train.shape
print train[:1]
train_new = SelectKBest(chi2, k=2).fit_transform(train, train_labels)
print train_new.shape
print train_new[:1]
test = test_features_nb
test_new = SelectKBest(chi2, k=2).fit_transform(test, test_labels)

(428L, 24L)
[[  5.14000000e+02   1.20000000e+02   2.11000000e+01   3.14000000e+00
    7.03000000e+00   4.47000000e-01   2.00000000e-02   1.10000000e-01
    1.82000000e-01   1.70000000e+00   2.43000000e+00   7.00000000e-01
    1.42000000e+00   2.93000000e+00   4.35000000e+00   8.90000000e-01
    6.90000000e-01   3.40000000e-01   2.41000000e+00   1.03000000e+00
    8.00000000e+00   8.00000000e+00   1.31000000e+02   2.06000000e+02]]
(428L, 2L)
[[ 514.  120.]]


In [130]:
#feature_list = [columns["GP"],columns["GS"],columns["MIN"],columns["PTS"],columns["FG%"],
#      columns["3P%"],columns["FT%"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

# Naive Bayes

In [282]:
clfNB = GaussianNB()

In [283]:
clf.fit(train_features_nb, train_labels)

GaussianNB()

In [284]:
predNB = clf.predict(test_features_nb)

In [285]:
accNB = metrics.accuracy_score(test_labels,predNB)
accNB
# first try, with all features - 66%
# second, GP, GS, PTS, FG%, 3P%, FT%, YR, POS, 3PM, FTM, FGM removed - 73%
# third - Univariate feature selection "Select K best" - 47% only two features GP i GS
# fourth - Univariate feature selection "SelectPercentile, f_classif" 74,6%

1.0

In [295]:
confm_nb = metrics.confusion_matrix(test_labels, pred)
confm_nb
# labels 1 - 2 - 3 - 4 - 5 

array([[13,  0,  0,  0,  0],
       [13, 15,  1,  0,  2],
       [ 0,  3, 13,  1,  1],
       [ 0,  0,  1, 16, 19],
       [ 0,  0,  0,  7, 37]])

# SVM

In [287]:
clfSVM= SVC()

In [288]:
clfSVM.set_params(kernel = 'linear')

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [289]:
clfSVM.fit(train_features_nb, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [290]:
predSVM = clfSVM.predict(test_features_nb)

In [291]:
accSVM = metrics.accuracy_score(test_labels,predSVM)
accSVM
# first try - 78% same features as nb
# second try - 82,4% kernel='linear'

0.99295774647887325

In [293]:
confm_nb = metrics.confusion_matrix(test_labels, predSVM)
confm_nb

array([[13,  0,  0,  0,  0],
       [ 0, 31,  0,  0,  0],
       [ 0,  0, 18,  0,  0],
       [ 0,  0,  0, 36,  0],
       [ 0,  0,  1,  0, 43]])

# Logistic Regression

In [302]:
lr_feature_list = [22]
train_features_lr, test_features_lr, val_features_lr = np_featuriser(train_set_nb, test_set_nb, val_set_nb,lr_feature_list)

lr_feature_list = [0, 1, 2, 3, 13, 18, 19, 20, 21, 22]
train_features_lr, test_features_lr, val_features_lr = np_featuriser(train_set_nb, test_set_nb, val_set_nb,lr_feature_list)


columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

In [303]:
#for i in remove:
#    print i
#    print columns.keys()[columns.values().index(i)]

In [296]:
logreg = linear_model.LogisticRegression(C=1e5)

In [304]:
logreg.fit(train_features_lr, train_labels)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [305]:
predLR = logreg.predict(test_features_lr)

In [306]:
accLR = metrics.accuracy_score(predLR, test_labels)
accLR
# first try 74,6%

0.88732394366197187